![data cube logo](https://static.wixstatic.com/media/8959d6_a13dc7ece5be4678af57c8f7c1667734~mv2.png/v1/fill/w_279,h_177,al_c,q_85,usm_0.66_1.00_0.01,enc_auto/VSDC_DataCubeSplash_6Partner_edited.png)
# Land Surface Temperature Product Testing

 - Sign up to the VA Datacube (INSERT LINK) to run this notebook interactively from a browser
 - <b>Compatibility</b>: Notebook currently compatible with both the NCI and DEA Sandbox environments    

 - <b>Products used</b>:
    
    products....
 
 - <b>Special requirements:</b> An optional description of any special requirements, e.g. If running on the NCI, ensure that
    module load otps is run prior to launching this notebook
 
 - <b>Prerequisites:</b> An <i>optional</i> list of any notebooks that should be run or content that should be understood
    prior to launching this notebook
 
 - <b>Notebook Use:</b> If you would like to modify and save this notebook, please copy and paste it into your own folder on the data cube, here is how:

 


## Background


An <i>optional</i> overview of the scientific, economic or environmental management issue or challenge being addressed by Digital Earth Australia. For Beginners_Guide or Frequently_Used_Code notebooks, this may include information about why the particular technique or approach is useful or required. If you need to cite a scientific paper or link to a website, use a persistent DOI link if possible and link in-text (e.g. Dhu et al. 2017).

## Description

A <i>compulsory</i> description of the notebook, including a brief overview of how Digital Earth Australia helps to address the problem set out above. It can be good to include a run-down of the tools/methods that will be demonstrated in the notebook:

1. First we do this
2. Then we do this
3. Finally we do this

---------------------------------------------------------------------------------------------------------------------------

## Getting started


Provide any particular instructions that the user might need, e.g. To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

### Load packages
Import Python packages that are used for the analysis.

Use standard import commands; some are shown below. Begin with any iPython magic commands, followed by standard Python packages, then any additional functionality you need from the Scripts directory.

In [1]:
%matplotlib inline

import datacube
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

from deafrica_tools.load_era5 import load_era5
from deafrica_tools.datahandling import load_ard, mostcommon_crs
from deafrica_tools.plotting import rgb

### Connect to the datacube

Connect to the datacube so we can access DEA data. The app parameter is a unique name for the analysis which is based on the notebook file name

In [2]:
dc = datacube.Datacube(app='LST_Testing')

### Analysis parameters
An optional section to inform the user of any parameters they'll need to configure to run the notebook:
 - param_name_1: Simple description (e.g. example_value). Advice about appropriate values to choose for this parameter.
 - param_name_2: Simple description (e.g. example_value). Advice about appropriate values to choose for this parameter.

 

In [3]:
param_name_1 = 'example_value'
param_name_2 = 'example_value'

## Heading 1

Use headings to break up key steps/stages of the notebook.

Use markdown text for detailed, descriptive text explaining what the code below does and why it is needed.

<b>Note</b>: Use this markdown format (sparingly) to draw particular attention to an important point or caveat

In [6]:
# List Landsat products available in DE Africa
dc_products = dc.list_products()
display_columns = ['name', 'description']
dc_products[dc_products.name.str.contains(
    '_st').fillna(
        False)][display_columns].set_index('name')

,description
name,
ls5_c2l2_st,USGS Landsat 5 Collection 2 Level-2 Surface Te...
ls5_c2l2_st_v2,USGS Landsat 5 Collection 2 Level-2 Surface Te...
ls7_c2l2_st,USGS Landsat 7 Collection 2 Level-2 Surface Te...
ls7_c2l2_st_v2,USGS Landsat 7 Collection 2 Level-2 Surface Te...
ls8_c2l2_st,USGS Landsat 8 Collection 2 Level-2 Surface Te...
ls8_c2l2_st_v2,USGS Landsat 8 Collection 2 Level-2 Surface Te...
ls9_c2l2_st,USGS Landsat 9 Collection 2 Level-2 Surface Te...
ls9_c2l2_st_v2,USGS Landsat 9 Collection 2 Level-2 Surface Te...


In [7]:
#List measurements
dc_measurements = dc.list_measurements()
dc_measurements.loc['ls7_c2l2_st']

,name,dtype,units,nodata,aliases,flags_definition
measurement,,,,,,
ST_B6,ST_B6,uint16,Kelvin,0,"[band_6, st, surface_temperature]",NaN
ST_TRAD,ST_TRAD,int16,W/(m2.sr.μm),-9999,"[trad, thermal_radiance]",NaN
ST_URAD,ST_URAD,int16,W/(m2.sr.μm),-9999,"[urad, upwell_radiance]",NaN
ST_DRAD,ST_DRAD,int16,W/(m2.sr.μm),-9999,"[drad, downwell_radiance]",NaN
ST_ATRAN,ST_ATRAN,int16,1,-9999,"[atran, atmospheric_transmittance]",NaN
ST_EMIS,ST_EMIS,int16,1,-9999,"[emis, emissivity]",NaN
ST_EMSD,ST_EMSD,int16,1,-9999,"[emsd, emissivity_stddev]",NaN
ST_CDIST,ST_CDIST,int16,Kilometers,-9999,"[cdist, cloud_distance]",NaN
QA_PIXEL,QA_PIXEL,uint16,bit_index,1,"[pq, pixel_quality]","{'snow': {'bits': 5, 'values': {'0': 'not_high..."


In [10]:
# Define the analysis region (Lat-Lon box)
lat = 37.208168 
lon = -80.443249 

#This is going to control how large the study area is. 
lon_buffer = 0.003
lat_buffer = 0.003

# Define the time window
time_range = ('2021-01', '2022-12')

# Combine central lat,lon with buffer to get area of interest
lat_range = (lat-lat_buffer, lat+lat_buffer)
lon_range = (lon-lon_buffer, lon+lon_buffer)

# Load land surface temperature and quality assessment
# We can use the alias names to call the bands
measurements = ['surface_temperature', 'surface_temperature_quality']
#measurements = ['thermal_radiance']


In [11]:


# Create a reusable query
query = {
    'y': lat_range,
    'x': lon_range,
    'time': time_range,
    'measurements': measurements,
    'resolution': (-10, 10),
    'output_crs': 'epsg:6933',
    'group_by':'solar_day'
}

# Load available data from Sentinel-2
ds = load_ard(
    dc=dc,
    products=['ls7_c2l2_st_v2'],
    mask_filters=[('opening', 3),('dilation', 3)],
    **query,
)

print(ds)


Using pixel quality parameters for USGS Collection 2
Finding datasets
    ls7_c2l2_st_v2


KeyError: 'surface_temperature_quality'

### Subheading 1

Use subheadings to break up steps within a single section.

In [ ]:
# Use code comments for low-level documentation of code
b = 2

## Heading 2

Use markdown text for detailed, descriptive text explaining what the code below does and why it is needed.


In [1]:
# Use code comments for low-level documentation of code
c = 3

## Additional information

<b>License:</b> The code in this notebook is licensed under the Apache License, Version 2.0. Digital Earth Australia data is licensed under the Creative Commons by Attribution 4.0 license.

<b>Contact:</b> If you need assistance, please post a question on the Open Data Cube Slack channel or on the GIS Stack Exchange using the open-data-cube tag (you can view previously asked questions here). If you would like to report an issue with this notebook, you can file one on Github.

<b>Last modified:</b> May 2020

<b>Compatible datacube version:</b>

In [ ]:
print(datacube.__version__)

## Tags 
Browse all available tags on the VA Data Cube User Guide's Tags Index 